In [2]:
URL = "https://www.cinepolis.com.ar/"

# Setup

In [147]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path="./chromedriver", options=options)

driver.get(URL)

In [146]:
driver.close()

# Process 1 movie

In [7]:
movies = driver.find_elements_by_xpath("//div[contains(@class, 'movie-grid')]/div/a")
movies_links = [m.get_attribute('href') for m in movies]
movies_links

['https://www.cinepolis.com.ar/peliculas/fate-stay-night-heavens-feel-parte-3',
 'https://www.cinepolis.com.ar/peliculas/godzilla-vs-kong',
 'https://www.cinepolis.com.ar/peliculas/la-noche-mas-larga',
 'https://www.cinepolis.com.ar/peliculas/los-intrusos',
 'https://www.cinepolis.com.ar/peliculas/la-fabrica-de-suenos',
 'https://www.cinepolis.com.ar/peliculas/calles-en-guerra',
 'https://www.cinepolis.com.ar/peliculas/corpus-christi',
 'https://www.cinepolis.com.ar/peliculas/mujer-maravilla-1984',
 'https://www.cinepolis.com.ar/peliculas/el-secreto-del-bosque',
 'https://www.cinepolis.com.ar/peliculas/la-noche-magica',
 'https://www.cinepolis.com.ar/peliculas/la-leyenda-de-la-viuda',
 'https://www.cinepolis.com.ar/peliculas/tom-y-jerry',
 'https://www.cinepolis.com.ar/peliculas/tenet',
 'https://www.cinepolis.com.ar/peliculas/trolls-2-gira-mundial',
 'https://www.cinepolis.com.ar/peliculas/malasana-32',
 'https://www.cinepolis.com.ar/peliculas/las-brujas']

In [8]:
link = movies_links[0]
link

'https://www.cinepolis.com.ar/peliculas/fate-stay-night-heavens-feel-parte-3'

In [9]:
driver.get(link)

In [10]:
title = driver.find_element_by_css_selector(".title > * > .title-text")
title.text

'FATE STAY NIGHT HEAVEN’S FEEL: PARTE 3'

In [12]:
synopsis = driver.find_element_by_css_selector("#sinopsis")
synopsis.text

'¡La historia de Fate Stay Night Heaven’s Feel llega a su fin!El capítulo final de la trilogía más ambiciosa y oscura de esta franquicia está aquí. Sakura Matou con los pecados que ha cometido se hunde en la oscuridad, Shirou Emiya ha jurado protegerla y detener la Guerra por el Santo Grial, una guerra dónde poderosos Magos y Espíritus Heroicos trabajan juntos para derrotar a los demás, el vencedor obtendrá el Santo Grial, un contenedor que puede hacer realidad cualquier deseo… Pero esta guerra ha corrompido al Grial. ¿Podrá Shirou salvar a Sakura? ¿Cuál será el precio de esto?'

In [191]:
html = driver.find_element_by_xpath("//*[@id='tecnicos']").get_attribute("innerHTML")

from scrapy.selector import Selector

fields_content = Selector(text=html).xpath("//p/text()").getall()
fields_content = [field.strip(': \n') for field in fields_content if field.strip()]

fields_name = Selector(text=html).xpath("//p/strong/text()").getall()

dict(zip(fields_name, fields_content))

{'Título Original': 'The widow',
 'Género': '',
 'Calificación': 'P-13',
 'Duración': '86 min.',
 'Distribuidora': 'BF Paris'}

In [144]:
origin, genres, director, actors, rated, duration = fields

print(f"""
Origin: {origin},
Genres: {genres},
Director: {director},
Actors: {actors},
Rated: {rated},
Duration: {duration}
""")


Origin: Japón,
Genres: Animación, Romance, Fantasía, Acción, Drama,
Director: Tomonori Sudô,
Actors: Noriaki Sugiyama, Noriko Shitaya, Ayako Kawasumi, Kana Ueda, Mai Kadowaki, Miki Itō, Jouji Nakata, Masane Tsukayama, Yū Asakawa, Tetsu Inada,
Rated: N/A,
Duration: 123 min.



# Process N movies

In [186]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from IPython.display import JSON

from scrapy.selector import Selector

def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    
    for a, b in replacements:
        s = s.lower().replace(a, b)
        
    return s

def get_technical_fields(x):
    html = x.find_element_by_xpath("//*[@id='tecnicos']").get_attribute("innerHTML")
    
    fields_content = Selector(text=html).xpath("//p/text()").getall()
    fields_content = [field.strip(': \n') for field in fields_content if field.strip()]
    
    fields_name = Selector(text=html).xpath("//p/strong/text()").getall()
    fields_name = map(normalize, fields_name)
    
    return dict(zip(fields_name, fields_content))


movies = driver.find_elements_by_xpath("//div[contains(@class, 'movie-grid')]/div/a")
movies_links = [m.get_attribute('href') for m in movies]

data = []

for link in movies_links:
    driver.get(link)
    
    WebDriverWait(driver, 2).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, ".title > * > .title-text"))
    )  
    
    technical_fields = get_technical_fields(driver)
        
    data.append({
        "title": driver.find_element_by_css_selector(".title > * > .title-text").text,
        "synopsis": driver.find_element_by_css_selector("#sinopsis").text,
        "origin": technical_fields.get('origen'),
        "genres": technical_fields.get('genero'),
        "director": technical_fields.get('director'),
        "actors": technical_fields.get('actores'),
        "rated": technical_fields.get('calificacion'),
        "duration": technical_fields.get('duracion'),
        "distributor": technical_fields.get('distribuidora'),
        "released": None,
        "source": "Cinépolis"
    })
    
JSON(data)

<IPython.core.display.JSON object>

In [11]:
%config Completer.use_jedi = False